<a href="https://colab.research.google.com/github/sooonsyk/ESAA-22-2/blob/main/W4_HW_mon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **앙상블 학습과 랜덤 포레스트**
- 무작위로 선택된 수천 명의 사람에게 복잡한 질문을 하고 대답을 모으는 것이 전문가의 답보다 나음 - 대중의 지혜
- 일련의 예측기(앙상블)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있음 - 앙상블 학습
  - 결정 트리의 앙상블 - 랜덤 포레스트

##**7.1 투표 기반 분류기**
- 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것, 다수결 투표로 정해짐
- 각 분류기가 약한 학습기일지라도 충분하게 많고 다양하다면 앙상블은 강한 학습기가 될 수 있음
  - 큰 수의 법칙 때문
  - 모든 분류기가 완변하게 독립적이고 오차에 상관관계가 없어야 가능 - 같은 데이터로 훈련시키기 때문에 가정에 맞지 않음
  - 분류기가 가능한한 서로 독립적일 대 최고의 성능 발휘 - 각기 다른 알고리즘으로 학습시키면 다양한 분류기 얻을 수 있음
    - 다른 종류의 오차를 만들어서 정확도 향상 시킴

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier # 사이킷런의 투표 기반 분류기
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [4]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.904
SVC 0.896
VotingClassifier 0.912


- 투표 기반 분류기가 성능 더 높음
- **간접 투표** :  모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_proba()메서드가 있으면) 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측 
  - 확률이 높은 투표에 비중을 두기 때문에 직접 투표 방식보다 성능이 높음
  - voting = "soft"로 바꾸고 모든 분류기가 클래스의 확률을 추정할 수 있으면 됨

## **7.2 배깅과 페이스팅**
- 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각각 다르게 학습시키는 것
- **배깅** : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- **페이스팅** : 중복을 허용하지 않고 샘플링하는 방식
- 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측 만듦
- 수집 함수는 전형적으로 분류일 때는 통계적 최빈값, 회귀에 대해서는 평균을 계산
- 개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만 수집 함수를 통과하면 편향과 분산이 모두 감소
- 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어듦

###**사이킷런의 배깅과 페이스팅**

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, #결정트리 분류기 500개의 앙상블
    max_samples = 100, bootstrap=True, #각 분류기는 훈련세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련 
    n_jobs=1 #CPU 코어수 지정, -1은 모든 코어 사용
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

- BaggingClassifier는 클래스 확률을 추정할 수 있으면 직접 투표 대신 간접 투표 방식 사용

###**7.2.2 oob 평가**
- 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링 되고 어떤 것은 전혀 선택되지 않을 수 있음 - 선택되지 않은 훈련 샘플을 oob 샘플이라고 부름, 예측기마다 다름
- 예측기가 훈련되는 동안에는 oob샘픔을 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플 사용해 평가할 수 잇음

In [7]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True) #oob_score=True 로 지정하면 훈련 끝난 후 자동으로 oob 평가 수행

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9013333333333333

In [9]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.896

In [10]:
bag_clf.oob_decision_function_
#oob 샘플에 대한 결정 함수 값 - 이 경우 각 훈련 샘플의 클래스 확률

array([[0.40437158, 0.59562842],
       [0.35675676, 0.64324324],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.09230769, 0.90769231],
       [0.36363636, 0.63636364],
       [0.01156069, 0.98843931],
       [0.96067416, 0.03932584],
       [0.95238095, 0.04761905],
       [0.76923077, 0.23076923],
       [0.00564972, 0.99435028],
       [0.78074866, 0.21925134],
       [0.83957219, 0.16042781],
       [0.97237569, 0.02762431],
       [0.07692308, 0.92307692],
       [0.        , 1.        ],
       [0.98958333, 0.01041667],
       [0.94623656, 0.05376344],
       [1.        , 0.        ],
       [0.0212766 , 0.9787234 ],
       [0.41317365, 0.58682635],
       [0.91176471, 0.08823529],
       [1.        , 0.        ],
       [0.9787234 , 0.0212766 ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.61797753, 0.38202247],
       [0.

##**7.3 랜덤 패치와 랜덤 서브스페이스**
- BaggingClassifier는 특성 샘플링 지원 - max_features, bootstrap_features 로 조절
- **랜덤 패치 방식** : 훈련 특성과 샘플을 모두 샘플링하는 것
- **랜덤 서브스페이스 방식** : 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것
  - bootstrap=False, max_samples=1.0, bootstrap_features=True, max_features 1.0보다 작게 설정
  - 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춤
  

##**7.4 랜덤 포레스트**
- 일반적으로 배깅 또는 페이스팅을 적용한 결정 트리 앙상블, 전형적으로 max_samples를 훈련 세트의 크기로 지정
- BaggingClassifier에 DecisionTreeClassifier를 넣어 만드는 대신 결정 트리에 최적화되어 사용하기 편한 DecisionTreeClassifier 사용 가능

In [12]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

- 랜덤 포레스트는 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입함 - 트리 더 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 더 훌륭한 모델 만들어냄

###**7.4.1 엑스트라 트리**
- **익스트림 랜덤 트리** : 극단적으로 무작위한 트리의 랜덤 포레스트
  - 모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되는 작업 중 하나이므로 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠름
  - 사이킷런의 ExtraTreesClassifier를 사용

###**7.4.2 특성 중요도**
- 랜덤 포레스트는 특성의 상대적 중요도 측정하기 쉬움
  - 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정
  - 훈련이 끝난ㄴ 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값 정규화함 - feature_importances_

In [13]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.11941625313301481
sepal width (cm) 0.020420019411930737
petal length (cm) 0.42380756675362147
petal width (cm) 0.436356160701433


##**7.5 부스팅**
- 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법
- 앞의 모델을 보완해나가면서 일련의 예측기를 학습시키는 것

###**7.5.1 에이다부스트**
- 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것 - 학습하기 어려운 샘플에 점점 더 맞춰지게 됨
- 각 예측기는 이전 예측기가 훈련되고 평가된 후에 학습될 수 있기 때문에 병렬화할 수 없음
- 새 예측기의 가중치가 계산되고 샘픔의 가중치를 업데이트 해서 또 다른 예측기를 훈련 시키는 식으로 반복
- 지정된 예측기 수에 도달하거나 완변한 예측기가 만들어지면 중지됨
- 예측할 때 단순히 모든 예측기의 예측을 게산하고 예측기 가중치를 더해 예측 결과를 만듦, 가중치 합이 가장 큰 클래스가 예측 결과가 됨
- 사이킷런은 SAMME 라는 에이다부스트의 다중 클래스 버전을 사용 - 클래스가 두 개일 때는 에이다부스트와 동일
  - 예측기가 클래스의 확률을 추정할 수 있다면 SAMME.R 사용

In [14]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, #200개의 아주 얕은 결정 트리
    algorithm='SAMME.R', learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

#에이다부스트의 기본 추정기
#과대적합시 추정기 수를 줄이거나 규제를 강하게

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

###**7.5.2 그레이디언트 부스팅**
- 이전까지의 오차를 보정하도록 순차적으로 추가하지만 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킴

In [22]:
#잡음이 섞인 2차 곡선 형태의 훈련 세트
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [23]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [24]:
#첫번째 예측기에서 생긴 잔여 오차에 두번째 훈련
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [25]:
#두번째 예측기가 만든 잔여 오차에 세번째 회귀 모델 훈련
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [26]:
import numpy as np
X_new = np.array([[0.8]])

In [27]:
#3개의 트리를 포함하는 앙상블 모델
#새로운 샘플에 대한 예측 만들려면 모든 트리의 예측 더함
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

- 사이킷런의 GreadientBoostingRegressor를 사용하면 간단하게 같은 앙상블 만들 수 있음

In [28]:
from sklearn.ensemble import GradientBoostingRegressor

grbt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0) 
grbt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

- learning_rate가 각 트리의 기여 정도 조절 - 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 예측의 성능은 좋아짐 - 축소 기법
- 너무 많으면 과대적합될 수 있음
- 최적의 트리 수 찾기 위해서 조기 종료 기법 사용 - staged_predict() 메서드 사용
  - 훈련의 각 단게에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자 반환

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=85)

- 많은 수의 트리를 먼저 훈련시키고 최적의 수를 찾기 위해 살펴보는 대신 실제로 훈련을 중지하는 방법으로 조기 종료 구현할 수 있음
  - warm_start=True로 설정하면 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 함

In [30]:
#연속해서 5번의 반복동안 검증 오차가 향상되지 않으면 훈련 멈춤
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up=0
for n_estimators in range(1, 120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up=0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break #조기 종료

- 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 subsample 매개변수 지원 - 편향이 높아지는 대신 분산 낮아짐, 훈련 속도 상당히 높임 - 확률적 그레이디언트 부스팅
- 최적화된 그레이디언트 부스팅 구현으로 XGBoost - 익스트림 그레이디언트 부스팅

In [31]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

[13:00:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [32]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2) #조기 종료 기능 제공
y_pred = xgb_reg.predict(X_val)

[13:01:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.275313
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.247499
[2]	validation_0-rmse:0.222947
[3]	validation_0-rmse:0.201049
[4]	validation_0-rmse:0.181641
[5]	validation_0-rmse:0.166735
[6]	validation_0-rmse:0.153621
[7]	validation_0-rmse:0.141686
[8]	validation_0-rmse:0.131314
[9]	validation_0-rmse:0.122047
[10]	validation_0-rmse:0.112234
[11]	validation_0-rmse:0.10366
[12]	validation_0-rmse:0.097232
[13]	validation_0-rmse:0.092018
[14]	validation_0-rmse:0.087123
[15]	validation_0-rmse:0.083105
[16]	validation_0-rmse:0.079681
[17]	validation_0-rmse:0.076621
[18]	validation_0-rmse:0.074141
[19]	validation_0-rmse:0.071896
[20]	validation_0-rmse:0.070115
[21]	validation_0-rmse:0.068281
[22]	validation_0-rmse:0.066894
[23]	validation_0-rmse:0.065783
[24]	validation_0-rmse:0.064704
[25]	validation_0-rm

##**7.6 스태킹**
- 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까?
- 마지막 예측기가 최종 예측 만듦 - 블렌더 혹은 메타 학습기
- 블렌더를 학습시키는 일반적인 방법은 홀드아웃 세트를 이용하는 것
  - 훈련 세트를 두 개의 서브셋으로 나눔
  - 첫번째 서브셋은 첫번째 레이어의 예측을 훈련시키기 위해 사용
  - 첫번째 레이어의 예측기를 사용해 두번째 (홀드아웃) 세트에 대한 예측 만듦
      - 예측기들이 훈련하는 동안 이 샘플들을 전혀 보지 못했기 때문에 이때 만들어진 예측은 완전히 새로운 것
  - 홀드아웃세트의 각 샘플에 대해 예측기 계수만큼 예측값이 있음
  - 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트 만듦 - 예측기 개수만큼의 차원 가짐
  - 블랜더가 새 훈련 세트로 훈련됨 - 첫번재 레이어의 예측을 가지고 타깃값을 예측하도록 학습됨
- 블렌더 여러 개 훈련 시키는 것도 가능 - 블렌더만의 레이어가 만들어짐
- 사이킷런은 스태킹 직접 지원하지는 않음
